In [1]:
import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import OpenAI

from website import Website

In [3]:
link_system_prompt = "You are provided with a list of links found on a Italian restaurant webpage. \
You are able to decide which of the links would be most relevant to include in the restaurant menu, \
such as links to an menu pdf file, Menù page, Piatti, or Bevande.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "menu pdf", "url": "https://www.ristoranteapprodo.com/Documenti/MenuEstivo2024.pdf"},
        {"type": "menu page", "url": "https://www.giocapizza.com/men%C3%B9"}
    ]
}
"""

In [4]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key) > 10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

MODEL = 'gpt-4o-mini'
openai = OpenAI()

ed = Website("https://www.giocapizza.com/")
print(ed.links)

API key looks good so far
['https://www.giocapizza.com', 'tel:349-6705657', 'https://www.instagram.com/giocapizza/', 'https://www.facebook.com/giocapizza/', 'https://www.tripadvisor.it/Restaurant_Review-g2337656-d17784755-Reviews-Gioca_Pizza-Adrara_San_Martino_Province_of_Bergamo_Lombardy.html', 'https://www.youtube.com/@GiocaPizza', 'https://www.pinterest.jp/giocapizza/', 'https://www.giocapizza.com', 'https://www.giocapizza.com/incorniciate', 'https://www.giocapizza.com/menù', 'https://www.giocapizza.com/servizi', 'https://www.giocapizza.com/menù', 'https://www.giocapizza.com/incorniciate', 'https://www.giocapizza.com/incorniciate', 'https://www.giocapizza.com/incorniciate', 'mailto:giocapizza@gmail.com', 'http://www.sinapsisnc.com']


In [5]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the italian restaurant website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for the restaurant menu, respond with the full https URL in JSON format."
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt


In [6]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)


In [8]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result


In [9]:
system_prompt = "You are an assistant that analyzes the contents of several menu pages from an italian restaurant website \
and creates restaurant menu with dishes and prices in Euro. Respond in markdown."

def get_restaurant_menu_user_prompt(company_name, url):
    user_prompt = f"You are looking at a restaurant called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a restaurant menu in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt


In [10]:
def create_restaurant_menu(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_restaurant_menu_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [11]:
create_restaurant_menu("La Cascina", "https://www.lacascinacredaro.it/")

Found links: {'links': [{'type': 'piatti', 'url': 'http://www.byserviziinternet.com/cascina/#piatti'}]}


# La Cascina – Ristorante Pizzeria Menu

## Antipasti (Starters)
- **Bruschetta al Pomodoro** - €5.00  
  Grilled bread topped with fresh tomatoes, garlic, and basil.

- **Crostini Toscani** - €7.00  
  Toasted bread with traditional chicken liver pâté.

- **Tagliere di Salumi** - €9.00  
  Selection of cured meats served with pickles and bread.

## Primi Piatti (First Courses)
- **Gnocchetti di Patate con Erbette** - €10.00  
  Potato gnocchi with a blend of seasonal greens.

- **Paccheri con Polipetti** - €12.00  
  Large tubular pasta with baby octopus in a tomato sauce.

- **Risotto ai Frutti di Mare** - €15.00  
  Arborio rice cooked with fresh seafood.

- **Tagliolini al Tartufo** - €14.00  
  Homemade tagliolini pasta with truffle sauce.

- **Zuppa di Cipolle** - €8.00  
  Traditional onion soup topped with melted cheese.

## Secondi Piatti (Main Courses)
- **Filetto di Manzo** - €18.00  
  Grilled beef fillet served with a side of seasonal vegetables.

- **Pollo alla Griglia** - €12.00  
  Grilled chicken breast served with rosemary potatoes.

- **Branzino al Forno** - €17.00  
  Oven-baked sea bass served with a lemon-herb sauce.

## Pizze (Pizzas)
- **Margherita** - €8.00  
  Classic pizza with tomato sauce, mozzarella, and basil.

- **Diavola** - €10.00  
  Spicy salami pizza with tomato sauce and mozzarella.

- **Funghi e Prosciutto** - €11.00  
  Pizza topped with mushrooms and ham.

- **Vegetariana** - €9.50  
  Mixed vegetable pizza with mozzarella.

## Dessert
- **Tiramisu** - €5.00  
  Classic coffee-flavored Italian dessert.

- **Panna Cotta** - €5.50  
  Creamy dessert served with berry sauce.

- **Gelato** - €4.00  
  Selection of homemade ice creams.

## Bevande (Beverages)
- **Acqua Naturale / Frizzante** - €2.50  
  Still or sparkling water.

- **Birra Artigianale** - €4.00  
  Local craft beer.

- **Vino della Casa** - €5.50 / glass  
  House wine selection.

For reservations or inquiries, please contact us at +39 035 936383.  
**Address:** Via L. Cadorna, 9, 24060 - Credaro (BG)  
**Closed on Wednesdays**.